# Getting started with Amazon Braket Hybrid Jobs

This tutorial shows how to run your first Amazon Braket Hybrid Job. To get started, we consider small circuits with only one qubit and one gate.


## Learning outcomes 
* Run your first Braket Job! 
* Understand how to run scripts or functions
* Monitor job status and view logs 
* Save results from a job
* Use Braket Jobs within a specific AWS session
* Run Braket Jobs on quantum hardware 
* Create a Braket Job using the Braket console
* Use Local Braket Jobs to quickly test code


## Algorithm script

To create a Braket Job, we first need a Python script that we want to run. In this example, it's contained in `algorithm_script.py`. The script is printed in the code cell below for convenience. 

As shown, each of our circuits has only one $X$ rotation gate with a random angle. The circuit is repeated five times with different random rotations. We write the algorithm script as we usually do, except that we do not specify the backend QPUs or simulators explicitly. Instead, it is provided through environment variables which are passed to the algorithm script when creating the job. 

In [1]:
%%script echo "This block is a copy of the algorithm script."

import os
import numpy as np

from braket.aws import AwsDevice
from braket.circuits import Circuit
from braket.jobs import save_job_result


print("Test job started!")

# Use the device declared in the creation script
device = AwsDevice(os.environ["AMZN_BRAKET_DEVICE_ARN"])

counts_list = []
angle_list = []
for _ in range(5):
    angle = np.pi * np.random.randn()
    random_circuit = Circuit().rx(0, angle)

    task = device.run(random_circuit, shots=100)
    counts = task.result().measurement_counts

    angle_list.append(angle)
    counts_list.append(counts)
    print(counts)

# Save the variables of interest so that we can access later
save_job_result({"counts": counts_list, "angle": angle_list})

print("Test job completed!")

This block is a copy of the algorithm script.


## Creating your job

In this notebook, we use <code>AwsQuantumJob.create</code> to create a Braket Job. When the Braket Job is created, it starts the job instance (based on EC2) and spins up a Docker container. The instance type, container, and other configurations can be specified via keyword arguments. See the [developer guide](https://docs.aws.amazon.com/braket/latest/developerguide/what-is-braket.html) and other example notebooks to learn more about how to customize your job.

Our example uses the following inputs for <code>AwsQuantumJob</code>:
- <b>device</b>: The arn of the Braket simulator or QPU we want to use. It will be passed as an environment variable to the algorithm script.
- <b>source_module</b>: The path to a file or a Python module that contains your algorithm script. It will be uploaded to the container for running the Braket Job.
- <b>wait_until_complete (optional)</b>: If true, the function call will wait until the Braket Job is completed and will additionally print logs to the local console. Otherwise, it will run asynchronously. 

In [2]:
from braket.aws import AwsQuantumJob

In [ ]:
# This cell should take about 5 mins
job = AwsQuantumJob.create(
    device="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="algorithm_script.py",
    wait_until_complete=True,
)

In this example, the algorithm is defined by a single file, so the <code>source_module</code> is <code>algorithm_script.py</code> itself. Depending on your application, there are other options for setting the source module. For example, if you wish to only execute a part of <code>algorithm_script.py</code> at the start of a Braket Job, you can package that part to be a <code>starting_function()</code>. Then assign the function as the entry point by add the <code>entry_point</code> input arguments.

In [4]:
source_module = "algorithm_script.py"
entry_point = "algorithm_script:starting_function"

If your algorithm script requires other dependency such as helper functions, you can put them all in one folder, say the <code>algorithm_folder</code>. The input arguments would then be

In [5]:
source_module = "algorithm_folder"
entry_point = "algorithm_folder.algorithm_script:starting_function"

## Tracking progress and results

The status of a Braket Job can be checked by calling <code>job.state()</code>. Once completed, the result can be retrieved using <code>job.result()</code>. Logs and metadata are also accessible via `job.logs()` and `job.metadata()`. If you lose the job variable, you can always retrieve it by the arn of the job through <code>job=AwsQuantumJob("your-job-arn")</code>. The arn of a job can be found in the Amazon Braket Console. 

In [6]:
job.state()

'COMPLETED'

In [8]:
results = job.result()  # will return once job.state() = "COMPLETE"
print("counts: {}".format(results["counts"]))
print("angles: {}".format(results["angles"]))

counts: [{'0': 30, '1': 70}, {'0': 94, '1': 6}, {'0': 65, '1': 35}, {'1': 67, '0': 33}, {'1': 100}]
angles: [-4.30719611295441, 0.4724370180628953, -1.2437930051491033, -4.428025559424561, 3.1125936045823086]


In [9]:
print(job.logs())
print(job.metadata())

In [ ]:
job.download_result()  # download job result to local directory

## Running on QPUs

With Braket Jobs you can run hybrid algorithms on all QPUs available through Amazon Braket. When you select a QPU as your device to run on, you job will have priority access for the duration of your job. Quantum tasks created as part of your job will be executed ahead of other tasks that might be queued up on the device. This reduces the risk of certain task being delayed or drifting calibrations on the device. 

We can seamlessly swap our SV1 simulator for a QPU by changing the device argument in `AwsQuantumJob.create()`. When you create the job, Amazon Braket will wait for the QPU to become available and then start the job. The  following cell creates the job asynchronously. You can check on the status in the console or by using <code>job.state()</code>.

In [ ]:
job = AwsQuantumJob.create(
    device="arn:aws:braket:::device/qpu/rigetti/Aspen-9",
    source_module="algorithm_script.py",
    wait_until_complete=False,
)

## AWS Sessions

We can customize the default location where Braket Jobs saves and loads results in Amazon S3 by providing the AWS session information. The name of the S3 bucket needs to start with "amazon-braket-".

In [10]:
from braket.aws import AwsSession
 
# Set Amazon S3 bucket
aws_session = AwsSession(default_bucket="amazon-braket-bucket-name")

To create a Braket Job with this S3 bucket, we pass `aws_session` as an argument to ` AwsQuantumJob.create()`

```
job = AwsQuantumJob.create(
    device="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="algorithm_script.py",
    aws_session=aws_session # using specific S3 bucket
)
```

## Creating a Braket Job from the Braket console
Besides creating a Braket Job programmatically using <code>AwsQuantumJob.create</code>, there is also an option to create a job on the Braket console. Follow [this link](https://998640816978-braket.us-west-2.console.aws-dev.amazon.com/braket/home#/job/create) to the "Create a new job" page. First, we need to give our new job a unique name. In the advance settings, we can choose the AWS role for this job and the S3 bucket where Amazon Braket will put the files associated with this job. <b>(TO-DO: update the link for job creation)</b>

<img src="console_figures/1-create.png" alt="Create job. See the console_figures folder if the figure not displayed." style="width: 600px;"/>

Second, we select the algorithm script for the job. The script can be uploaded from a local console as a single script file. If there are many files, such as helper functions, that go with the algorithm script, we upload the files to a S3 bucket and provide the path of the bucket.
<img src="console_figures/2-algorithm.png" alt="Select algorithm script. See the console_figures folder if the figure not displayed." style="width: 600px;"/>

Third, you select a container environment for your job. The "Base" container is enough for the example in this notebook. For information about using other pre-built or custom containers, see the QAOA and the BYOC example notebook for Braket Job.
<img src="console_figures/3-container.png" alt="Select a container. See the console_figures folder if the figure not displayed." style="width: 600px;"/>

Next, you select a Braket managed simulator or QPU for your job and configure the execution settings. Finally, there are optional hyperparameter and data inputs, as well as the possibility to customize the default locations for checkpoints and output data. We will dive deeper into these advanced use cases in other example notebooks. After finishing all the settings, you create the job by clicking the "create job" button. We can now view the progress in the Braket console.
<img src="console_figures/4-execution.png" alt="Configuration for execution. See the console_figures folder if the figure not displayed." style="width: 600px;"/>

## Local Braket Jobs

For faster testing and debugging of your code, you can run a job locally in your own environment. This feature requires Docker to be installed in your local environment. Amazon Braket Notebook Instances have Docker pre-installed, so you can test local jobs in hosted notebooks instantly. To install Docker in your local environment, follow these [instructions](https://docs.docker.com/get-docker/). When a local job is created for the first time, it will take longer because it needs to build the container. The subsequent runs will be faster. Note, that local jobs will not be visible in the Amazon Braket Console.

To run a job in the local mode, simply create a <code>LocalQuantumJob</code> instead of <code>AwsQuantumJob</code>. Local jobs always run synchronously and display the logs. 

In [ ]:
from braket.jobs.local.local_job import LocalQuantumJob

# This cell should take about 2 min
job = LocalQuantumJob.create(
    device="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="algorithm_script.py",
)

## Summary

In this tutorial, we have created our first Braket Job with a simple batch of five circuits using the Amazon Braket SDK and, as an alternative, from the Braket console. We learned how to change the Amazon S3 bucket and the AWS region for a job, and how to save results. We learned about running on simulators and QPUs. Lastly, we used local jobs to quickly test code.